In [40]:
println("ready")

ready


# -3) vérifier que le spark context est disponible

In [41]:
sc

org.apache.spark.SparkContext@7ba72055

# -2) imports

In [42]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf

# -1) refaire un spark context personnalisé

In [43]:
val appName = "rennes"
val master  = "local[3]"
val conf    = new SparkConf()
conf.setAppName(appName)
conf.setMaster(master)


org.apache.spark.SparkConf@1ddd9406

In [44]:
sc.stop()
val sc = new SparkContext(conf)

In [45]:
sc.master

local[3]

# 1) Créer une première rdd

## 1.1) lecture d'un fichier

In [46]:
val cyclistes = sc.textFile("./logs_backup/cycliste_cyclistes.csv")

In [47]:
cyclistes.count()

1764

In [48]:
cyclistes.take(2)

Array(cycliste_azetu,6,31,femme,8.11111516393,21.9593538999,100, cycliste_aztv4,2,19,femme,10.5299319612,22.1102025653,8)

In [49]:
val velos = sc.textFile("./logs_backup/velos_etats.csv")

In [50]:
velos.take(1)

Array(velo_azem4,1485439420.58,azf38,0.95,97.3984151307)

# 1.2) filtrer

In [51]:
val velos2 = velos.filter(!_.isEmpty() )

##### Remarque : en scala il faut faire précéder les chaines par la lettre [s] pour pouvoir afficher la valeur de variables

In [52]:
val nb_lignes = velos.count()
println(s"fichier = ${velos.count()}, lignes non vides =  ${velos2.count()}")

fichier = 128668, lignes non vides =  64334


# 2) convertir un fichier en table sql

## 2.0) importer les librairies sql

In [53]:
import org.apache.spark.sql.Row;
import org.apache.spark.sql.types.{StructType, StructField, StringType};

# 2.1) créer un sqlContext

In [54]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

In [77]:
sqlContext.tableNames()

Array(velos)

# 2.2) définir le schéma de la table vélo

In [72]:
var champs = List(  StructField("nom"          , StringType, true),
                    StructField("time"         , StringType, true),
                    StructField("station"      , StringType, true),
                    StructField("performance"  , StringType, true),
                    StructField("nb_km_trajet" , StringType, true))
var schema = StructType(champs)

## 2.3) Transformer les lignes en tableau d'objet spécifiques : les 'Row'

In [57]:
val rowRDD = velos2.map(_.split(",")).map(champs => Row(champs(0), champs(1), champs(2),champs(3),champs(4)))

# 2.4) créer une DataFrame via le sqlContext

In [58]:
val veloDF = sqlContext.createDataFrame(rowRDD, schema)

# 2.5) Enregistrer la DataFrame en tant que table

In [136]:
val table_velos = "velos"
veloDF.registerTempTable(table_velos)

# 2.6) Requêter la table avec du SQL

##### requete 01 : compte le nb de lignes

In [137]:
val requete = s"Select count(*) from ${table_velos}"
sqlContext.sql(requete).show()

+--------+
|count(1)|
+--------+
|   64334|
+--------+



##### requete 02 : nb de nom distincts

In [139]:
val requete = s"Select count( distinct nom) from ${table_velos}"
sqlContext.sql(requete).show()

+-------------------+
|count(DISTINCT nom)|
+-------------------+
|                760|
+-------------------+



##### requete 03 : performance moyenne

In [140]:
// performance moyenne : 
val requete = s"Select mean( performance) from ${table_velos}"
sqlContext.sql(requete).show()

+--------------------------------+
|avg(CAST(performance AS DOUBLE))|
+--------------------------------+
|              0.7615443430900648|
+--------------------------------+



##### requete 04 : multi champs

In [141]:
val requete = s"""
Select nom                                       , 
       min   ( nb_km_trajet     )                , 
       mean  ( nb_km_trajet     ) as moyenne     ,  
       max   ( nb_km_trajet     )                ,
       sum   ( nb_km_trajet     ) as somme       ,
       count ( distinct station ) as nb_station
       
from ${table_velos}
group by nom
"""
sqlContext.sql(requete).show()

[Stage 140:====================================================>(198 + 2) / 200]+----------+-----------------+------------------+-----------------+------------------+----------+
|       nom|min(nb_km_trajet)|           moyenne|max(nb_km_trajet)|             somme|nb_station|
+----------+-----------------+------------------+-----------------+------------------+----------+
|velo_aeyi6|    106.124133158|  250.712765889171|    98.2821292635|25321.989354806272|        46|
|velo_azow8|    1000.72528089| 640.1191001270846|    994.915022987|129304.05822567109|        60|
|velo_aeqx7|    49.6741505596|     49.6741505596|    49.6741505596|     49.6741505596|         1|
|velo_aeufh|     102.45730718|  364.425467797552|    95.8711122327| 36806.97224755275|        48|
|velo_arod5|    101.702990964|352.68417590080105|    90.3094170541| 35621.10176598091|        48|
|velo_artu7|    100.583333498|374.04519581147616|    96.9156508692|37778.564776959094|        60|
|velo_azdf6|    109.735413268|     109

In [64]:
//import org.joda.time.{DateTimeZone}
//import org.joda.time.format.DateTimeFormat

##### requete 05 : conversion de timestamp en date

In [142]:
val requete = s"""
Select  nom,
        min(from_unixtime(time,'YYYY-MM-dd HH:mm:ss')) as premiere_sortie, 
        max(from_unixtime(time,'YYYY-MM-dd HH:mm:ss')) as derniere_sortie                
        from ${table_velos}
        group by nom
"""
sqlContext.sql(requete).show()

+----------+-------------------+-------------------+
|       nom|    premiere_sortie|    derniere_sortie|
+----------+-------------------+-------------------+
|velo_aeyi6|2017-01-26 14:08:41|2017-01-26 14:28:02|
|velo_azow8|2017-01-26 14:11:32|2017-01-26 14:38:02|
|velo_aeqx7|2017-01-26 14:03:22|2017-01-26 14:03:22|
|velo_aeufh|2017-01-26 14:06:59|2017-01-26 14:25:08|
|velo_arod5|2017-01-26 14:06:43|2017-01-26 14:22:34|
|velo_artu7|2017-01-26 14:06:43|2017-01-26 14:17:00|
|velo_azdf6|2017-01-26 14:03:43|2017-01-26 14:03:43|
|velo_aeqf9|2017-01-26 14:06:43|2017-01-26 14:29:22|
|velo_aerys|2017-01-26 14:11:29|2017-01-26 14:21:55|
|velo_aeyiq|2017-01-26 14:06:59|2017-01-26 14:14:36|
|velo_aelv1|2017-01-26 14:06:48|2017-01-26 14:20:03|
|velo_azqm1|2017-01-26 14:06:48|2017-01-26 14:25:03|
|velo_aztx1|2017-01-26 14:06:58|2017-01-26 14:10:09|
|velo_aei57|2017-01-26 14:07:53|2017-01-26 14:20:23|
|velo_arpjv|2017-01-26 14:06:44|2017-01-26 14:18:25|
|velo_azkw8|2017-01-26 14:13:48|2017-01-26 14:

##### requete 06 : imbrication de requêtes 

In [144]:
val requete = s"""
Select  nom, 
        from_unixtime(derniere_sortie - premiere_sortie, 'HH:mm:ss') as duree
from
(       Select  nom,
                min(time) as premiere_sortie, 
                max(time) as derniere_sortie                
        from ${table_velos}
        group by nom
        )
"""
sqlContext.sql(requete).show()

+----------+--------+
|       nom|   duree|
+----------+--------+
|velo_aeyi6|00:19:20|
|velo_azow8|00:26:29|
|velo_aeqx7|00:00:00|
|velo_aeufh|00:18:09|
|velo_arod5|00:15:51|
|velo_artu7|00:10:16|
|velo_azdf6|00:00:00|
|velo_aeqf9|00:22:39|
|velo_aerys|00:10:26|
|velo_aeyiq|00:07:37|
|velo_aelv1|00:13:14|
|velo_azqm1|00:18:14|
|velo_aztx1|00:03:10|
|velo_aei57|00:12:30|
|velo_arpjv|00:11:40|
|velo_azkw8|00:12:51|
|velo_ars89|00:00:00|
|velo_aeqxn|00:08:05|
|velo_artm0|00:00:00|
|velo_artsd|00:00:00|
+----------+--------+
only showing top 20 rows



# 3) reading csv

In [233]:
var champs         = List(  StructField("nom"    , StringType, true) ,
                            StructField("heure"  , StringType, true) ,
                            StructField("velo"   , StringType, true) ,
                            StructField("action" , StringType, true) )
var schema         = StructType(champs)
val DataSet_prise  = sqlContext.read.schema(schema).csv("./logs_backup/cycliste_prises.csv")
val table_prises   = "prises"
DataSet_prise.registerTempTable(table_prises)


In [239]:
var champs = List(  StructField("cyclise"  , StringType, true),
                    StructField("heure"    , StringType, true),
                    StructField("rendu"    , StringType, true),
                    StructField("duree"    , StringType, true),
                    StructField("velo"     , StringType, true))
var r_schema = StructType(champs)
val DataSet_rendu = sqlContext.read.schema(r_schema).csv("./logs_backup/cycliste_rendu.csv")
val table_rendu = "rendus"
DataSet_rendu.registerTempTable(table_rendu)

In [242]:
val requete = s"""  SELECT distinct  p.nom     , 
                                     p.heure   , 
                                     r.heure   ,
                                     r.rendu   ,
                                     r.duree   
                    FROM        prises AS p 
                    INNER JOIN  rendus AS r 
                    ON          p.velo = r.velo
"""
sqlContext.sql(requete).show()

+--------------+-------------+-------------+-------------+-------------+
|           nom|        heure|        heure|        rendu|        duree|
+--------------+-------------+-------------+-------------+-------------+
|cycliste_aze10|1485439602.26|1485440311.39| 1485440312.6|11.1029811867|
|cycliste_aze10|1485439630.58|1485440148.91|1485440149.29|9.76041518677|
|cycliste_aze10|1485439745.23|1485439681.31|1485439681.89|12.8273055141|
|cycliste_aze10|1485439757.55|1485439623.96|1485439624.41|7.85567722207|
|cycliste_aze10|1485439783.18|1485439938.97| 1485439939.1|3.96409418747|
|cycliste_aze10|1485439783.18|1485440619.75|1485440619.99|4.71465658798|
|cycliste_aze10| 1485439795.9|1485439693.79|1485439693.79|            0|
|cycliste_aze10|1485439808.25|1485439902.99|1485439903.27|8.24423764551|
|cycliste_aze10|1485439834.83|1485439727.11|1485439727.79|6.21738885458|
|cycliste_aze10|1485439847.64|1485439968.62|1485439968.82| 6.0272283344|
|cycliste_aze10|1485439860.77|1485439653.45|1485439

In [216]:
val requete = s"""
Select distinct p.nom, p.velo, p.action, p.heure, r.action, r.heure
from    prises as p
Join    rendus as r
ON p.velo == r.velo
"""

sqlContext.sql(requete).show()

Name: org.apache.spark.sql.AnalysisException
Message: cannot resolve '`p.heure`' given input columns: [action, nom, nom, heure , heure , velo, velo, action]; line 2 pos 41;
'Distinct
+- 'Project [nom#1869, velo#1871, action#1872, 'p.heure, action#1881, 'r.heure]
   +- Join Inner, (velo#1871 = velo#1880)
      :- SubqueryAlias p
      :  +- SubqueryAlias prises
      :     +- Relation[nom#1869,heure #1870,velo#1871,action#1872] csv
      +- SubqueryAlias r
         +- SubqueryAlias rendus
            +- Relation[nom#1878,heure #1879,velo#1880,action#1881] csv

StackTrace: 'Distinct
+- 'Project [nom#1869, velo#1871, action#1872, 'p.heure, action#1881, 'r.heure]
   +- Join Inner, (velo#1871 = velo#1880)
      :- SubqueryAlias p
      :  +- SubqueryAlias prises
      :     +- Relation[nom#1869,heure #1870,velo#1871,action#1872] csv
      +- SubqueryAlias r
         +- SubqueryAlias rendus
            +- Relation[nom#1878,heure #1879,velo#1880,action#1881] csv

  at org.apache.spark.sql.cat

In [157]:
DF_prise.getClass()

class org.apache.spark.sql.Dataset

In [156]:
DF_prise.map(champs => Row(champs(0), champs(1), champs(2),champs(3)))

Name: Unknown Error
Message: <console>:82: error: Unable to find encoder for type stored in a Dataset.  Primitive types (Int, String, etc) and Product types (case classes) are supported by importing spark.implicits._  Support for serializing other types will be added in future releases.
       DF_prise.map(champs => Row(champs(0), champs(1), champs(2),champs(3)))
                   ^
StackTrace: 